# Streaming Training Tweets Using Tweepy

In [19]:
import tweepy
import json
from pymongo import MongoClient

class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def on_connect(self):
        """Called when the connection is made"""
        
        print("You're connected to the streaming server.")

    def on_error(self, status_code):
        """This is called when an error occurs"""
        
        print('Error: ' + repr(status_code))
        return False

    def on_data(self, data):
        """This will be called each time we receive stream data"""
        
        client = MongoClient()

        # I stored the tweet data in a database called 'training_tweets' in MongoDB, if 
        # 'training_tweets' does not exist it will be created for you.
        db = client.training_tweets

        # Decode JSON
        datajson = json.loads(data)

        # I'm only storing tweets in English. I stored the data for these tweets in a collection 
        # called 'training_tweets_collection' of the 'training_tweets' database. If 
        # 'training_tweets_collection' does not exist it will be created for you. 
        if "lang" in datajson:
            db.training_tweets_collection.insert_one(datajson)


if __name__ == "__main__":

    # These are provided to you through the Twitter API after you create a account
    consumer_key = "owL3x8WeEke0BCHCQHUvEykKv"
    consumer_secret = "gam4Ha8K2UOOgisXxSIU35vAnmNR0DXDm2ILtnOJsROOIOPyVY"
    access_token = "1225979021344878594-B2jw0KZO5xDddPV5Vn8sQOuOvfCzwc"
    access_token_secret = "Ir0Lg9rwBzFTZd0Zy62Lb3sE14SrrjYdIKBeqoNB5OXUt"

    auth1 = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth1.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the 
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box. 
    LOCATIONS = [-19.0,-35.5,52.0,36.6,            # Africa
                 59.7,-11.2,150.1,74.7,            # Asia
                 -11.3,36.5,42.8,71.5,             # Europe
   #             107.2,-44.2,-179.2,-10.0,         # Oceania                    
                 -171.24,12.66,-51.92,70.9,        # North America
                 -82.8,-56.4,-34.2,13.1]           # South America
                        
    #LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472,        # Contiguous US
    #             -164.639405, 58.806859, -144.152365, 71.76871,         # Alaska
    #             -160.161542, 18.776344, -154.641396, 22.878623]        # Hawaii

    stream_listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True))
    stream = tweepy.Stream(auth=auth1, listener=stream_listener)
    stream.filter(locations=LOCATIONS)

You're connected to the streaming server.


KeyboardInterrupt: 

In [20]:
client = MongoClient()
db = client.training_tweets

In [30]:
for i in db.training_tweets_collection.find_one({"lang":{"$exists":"true"}}):
    print(i)

_id
created_at
id
id_str
text
display_text_range
source
truncated
in_reply_to_status_id
in_reply_to_status_id_str
in_reply_to_user_id
in_reply_to_user_id_str
in_reply_to_screen_name
user
geo
coordinates
place
contributors
is_quote_status
quote_count
reply_count
retweet_count
favorite_count
entities
extended_entities
favorited
retweeted
possibly_sensitive
filter_level
lang
timestamp_ms
